## Python Decorator
플라스크, 셀러리와 같은 파이썬 오픈 소스 라이브러리들을 보면 @route, @task 등의 구문을 사용합니다.  
대체 이 골뱅이 표기들은 무엇이고, 어떤 역할을 하길래 많이 사용되는 것일까요?  
이 골뱅이들의 이름은 데코레이터입니다. 장식해주는 것? 이름만 들어서는 감이 잘 오지 않습니다.  
먼저 예시를 한번 볼까요?  

데코레이터 함수는 우선 장식을 해줄 func을 파라미터로 전달받습니다.  
그리고 decorated라는 함수를 내부에 만들어주고 이를 리턴해줍니다.  
decorated 함수 안에서는 전달받은 함수의 앞 뒤에 실행할 코드를 작성해줍니다

In [8]:
def hello_decorator(func):
    def decorated():
        print('Hello!')
        func()
        print('Nice to meet you!')
    return decorated

In [9]:
@hello_decorator
def hello():
    print("I'm Jeff")
hello()

Hello!
I'm Jeff
Nice to meet you!


보셨나요? I'm Jeff 만 출력하는 함수가 데코레이터의 장식을 받아 앞 뒤로 문장을 출력합니다.  
**즉, 데코레이터는 장식시킬 함수의 앞과 뒤에서 코드를 실행합니다.**  
그렇다면 이런 데코레이터는 언제 사용하나요?  
바로 여러 함수들이 부분적으로 중복되는 작업을 수행하는 경우, 코드의 재사용성을 높이기 위해 사용합니다.  
예시를 통해서 알아보겠습니다.

In [13]:
import time

def task1():
    time.sleep(3)
    print('hello! this is task 1')

def task2():
    time.sleep(5)
    print('hello! this is task 2')

def task3():
    time.sleep(2)
    print('hello! this is task 3')

위 사례처럼 각기 다른 동작을 수행하며, 소요 시간이 다른 세 임의의 함수가 있다고 가정합니다.  
이 때 개발자는 각각의 함수가 소요 시간이 얼마나 걸리는지 확인하고자 합니다.  
그냥 단순히 코딩을 한다면 다음과 같이 짤 수 있습니다.

In [11]:
import time
from datetime import datetime

def task1():
    start = datetime.now()
    time.sleep(3)
    print('hello! this is task 1')
    end = datetime.now()
    print('timecost', end - start)

def task2():
    start = datetime.now()
    time.sleep(5)
    print('hello! this is task 2')
    end = datetime.now()
    print('timecost', end - start)

def task3():
    start = datetime.now()
    time.sleep(2)
    print('hello! this is task 3')
    end = datetime.now()
    print('timecost', end - start)

In [12]:
task1()
task2()
task3()

hello! this is task 1
timecost 0:00:03.002614
hello! this is task 2
timecost 0:00:05.000760
hello! this is task 3
timecost 0:00:02.002568


수행하고자 하는 동작 앞, 뒤에 현재 시간을 재는 코드가 추가되었습니다.  
그리고 수행을 완료한 이후 소요 시간을 출력해줍니다.  
그 결과로 서로 다른 세 함수에 동일한 코드가 삽입되었습니다.  

코드의 가독성 측면에서 이는 그다지 바람직하지 못합니다.  
데코레이터를 사용하여서 이를 해결해보겠습니다.  

In [22]:
from datetime import datetime

def timecost(func):
    def decorated():
        start = datetime.now()
        func()
        end = datetime.now()
        print('timecost', end - start)
    return decorated

In [23]:
import time

@timecost
def task1():
    time.sleep(3)
    print('hello! this is task 1')

@timecost
def task2():
    time.sleep(5)
    print('hello! this is task 2')

@timecost
def task3():
    time.sleep(2)
    print('hello! this is task 3')

In [24]:
task1()
task2()
task3()

hello! this is task 1
timecost 0:00:03.001846
hello! this is task 2
timecost 0:00:05.000390
hello! this is task 3
timecost 0:00:02.003846


어떤가요? 훨씬 깔끔해보이지 않나요?  
그런데 어째 함수 안에 함수가 들어있는 모양새가 그렇게 보기 좋진 않습니다.  
데코레이터를 함수가 아니라 클래스로 만들어보겠습니다.

In [33]:
from datetime import datetime

class Timecost():
    def __init__(self, func):
        self.func = func
        
    def __call__(self):
        start = datetime.now()
        self.func()
        end = datetime.now()
        print('timecost', end - start)

In [34]:
import time

@Timecost
def task1():
    time.sleep(3)
    print('hello! this is task 1')

@Timecost
def task2():
    time.sleep(5)
    print('hello! this is task 2')

@Timecost
def task3():
    time.sleep(2)
    print('hello! this is task 3')

In [35]:
task1()
task2()
task3()

hello! this is task 1
timecost 0:00:03.004501
hello! this is task 2
timecost 0:00:05.004979
hello! this is task 3
timecost 0:00:02.001314


이제 데코레이터가 무엇인지 감이 좀 오시나요?  
조금 더 나아가서 실제 프레임워크들에서는 데코레이터를 어떻게 사용하는지 알아보겠습니다.

In [27]:
from flask import Flask

app = Flask(__name__)


@app.route('/')
def hello_world():
    return 'Hello World!'

이는 파이썬 웹 어플리케이션 프레임워크인 플라스크에서 API를 작성하는 코드입니다.  
사용자는 자신이 수행하고자하는 동작 (Hello World 리턴)을 입력한 뒤에 데코레이터를 달아줍니다.  
플라스크 앱의 route 데코레이터 함수는 이제 / 라는 url로 http 요청이 들어오면 사용자가 작성한 함수를 실행합니다.  
그리고 이를 http response로 만들어서 요청자에게 반환시켜줍니다.  

In [28]:
from celery import Celery

app = Celery('tasks', broker='pyamqp://guest@localhost//')

@app.task
def add(x, y):
    return x + y

다음은 파이썬 비동기 분산처리 프레임워크 셀러리에서 비동기 처리를 해주는 코드입니다. (구체적인 내용은 몰라도 상관없습니다.)  
단순히 x와 y를 더해주는 함수에 app.task 함수를 데코레이터로 붙여줍니다.  
그러면 브로커에 연결된 샐러리 앱이 브로커 내부의 큐에 테스크가 쌓이는 지를 확인하고  
테스트가 있으면 가져와서 계산하고 다시 반환을 어쩌고 저쩌고 하는  
듣기만해도 현기증 나는 작업들을 해주는 멋진 비동기 함수로 재탄생합니다.  

**즉, 이러한 프레임워크의 강력한 기능들을 데코레이터를 통해 손쉽게 가져다 쓸 수 있는 것입니다.**

자, 이제 데코레이터가 무엇이고 실제 개발에선 어떻게 사용되는지 아시겠나요?  
끝으로 3줄 요약하자면 이렇습니다.  
1. 데코레이터는 함수를 파라미터로 전달 받으며, 그 함수 앞 뒤에 다른 동작들을 수행해준다.
2. 코드의 재사용성, 가독성을 높이기 위해 사용한다.
3. 프레임워크를 사용할 때 많이 볼 수 있다.


감사합니다!